# Yolo ile video işleme

In [ ]:
!pip install ultralytics
!pip install opencv-python-headless
!pip install matplotlib


In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO


In [ ]:
model = YOLO('yolov8n.pt')  # En küçük model olan YOLOv8n'yi kullanıyoruz


In [ ]:
from google.colab import files

uploaded = files.upload()  # Burada video dosyasını seçin ve yükleyin


In [ ]:
def detect_faces_in_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Video yazıcı ayarları
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLOv8 modelini kullanarak yüz tespiti yap
        results = model(frame)[0]

        # Tespit edilen yüzleri çerçeve içine al
        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            class_name = results.names[int(class_id)]
            if class_name == 'person':  # YOLOv8n modelinde 'person' olarak tanımlanmışsa
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{class_name} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Görüntüyü yaz
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Yüklediğiniz video dosyasının ismini buraya girin
video_filename = list(uploaded.keys())[0]
detect_faces_in_video(video_filename)


In [ ]:
from IPython.display import HTML
from base64 import b64encode

def display_video(file_path):
    video_file = open(file_path, "rb").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width=600 controls><source src="{video_url}" type="video/mp4"></video>""")

# Çıktı videosunu göster
display_video('output.avi')


In [ ]:
def detect_faces_from_camera():
    cap = cv2.VideoCapture(0)  # Kamerayı aç
    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # YOLOv8 modelini kullanarak yüz tespiti yap
        results = model(frame)[0]

        # Tespit edilen yüzleri çerçeve içine al
        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            class_name = results.names[int(class_id)]
            if class_name == 'person':  # YOLOv8n modelinde 'person' olarak tanımlanmışsa
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{class_name} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Görüntüyü göster
        cv2.imshow('YOLOv8 Face Detection', frame)

        # 'q' tuşuna basıldığında çık
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Kamera akışında yüz tespiti yap
detect_faces_from_camera()
